In [1]:
import pandas as pd
import requests as re
import ast
from pathlib import Path



This code extracts raw unprepared statistics for all matches from given competition and saves it into csv-file. You need Cyanide API key for this. To get the key contact Cyanide Studio Team com@cyanide-studio.com. I keep my private key in txt-file (it looks like https://web.cyanide-studio.com/ws/?bb=3&key=MYPRIVATEKEY)

In [2]:
with open('api_key.txt', 'r') as file:
    lines = file.readlines()
api_key = str(lines[0]).split('=')[-1]

For extracting data you need competition_id (competition_name may be not unique). If you want to get matches from official ladder (like I do), you may execute following code:
```python
url = f'https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}'

response = re.get(url)

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        for row in df[['id', 'name']].itertuples():
            print(f"{row.Index}: id={row.id}, name={row.name}")
    else:
        print("No competitions in this league.")
else:
    print(f'Error: {response.status_code}')
```
and choose id.

If you want to get matches from another league, you need its ID or name. Try something like this:
```python
league_name = 'Russian Blood Bowl League'
url = f"https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}&league_name={league_name}"

response = re.get(url)

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        for row in df[['id', 'name']].itertuples():
            print(f"{row.Index}: id={row.id}, name={row.name}")
    else:
        print("No competitions in this league.")
else:
    print(f'Error: {response.status_code}')
```

Besides competition_id you have to define several parameters: 
- league_name: league your competition is from (default is Official League);
- platform: doesn't matter much, used for naming saved csv files. I use "pc", "playstation", "xbox" or "crossplay";     
- chunk_limit: defines amount of rows you'll get for 1 iteration, 1000 is max;  
- request_limit: defines max amount of requests you code will make, current limits are 1000 per hour and 10000 per day, you have not to exceed these limits.

In [25]:
competition_id = '00000000-0000-0000-0000-000000000082'
league_name = 'Official League'
platform = 'crossplay'
chunk_limit = 1000
request_limit = 200

Execute following cell to get competition statistics. If something breaks mid-process, just execute cell again (if will save partially extracted data to temporary file and just continue from that point).

In [26]:
url = f"https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}&league_name={league_name}&limit=1000"
response = re.get(url)
break_flag = 0 # If we need to break while cycle
continue_flag = 0 # If we restart with previously saved data

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        
        if competition_id in df.id.values:
            competition_start = df[df.id == competition_id].date_created.values[0]
            competition_name = df[df.id == competition_id].name.values[0].replace(' ', '_')

            if Path('temporary.csv').exists():
                df = pd.read_csv('temporary.csv')
                n = 0
                print('Continue from previously loaded data.')
                continue_flag = 1
    
            else:
                print('Start extracting data.')
                url = f'''https://web.cyanide-studio.com/ws/bb3/matches/?key={api_key}\
&competition_id={competition_id}\
&start={competition_start}\
&limit={chunk_limit}'''
                n = 1
                response = re.get(url)

                if response.status_code == 200:
                    print(f'{n} request: OK')
                    data = response.json()
                    df = pd.json_normalize(data['matches'])

                else:
                    print(f'{n} request: ERROR: {response.status_code}')
                    df = pd.DataFrame()

            try:
                anchor_date = df.started.min()
                combined_df = df.copy()

            except Exception as e:
                print(f"Something broke: {e}. May be, no matches in this competition.")
                break_flag = 1

            try:
                while (len(df) == chunk_limit and n < request_limit) or continue_flag:
                    
                    if len((df[df.started != anchor_date])) == 0 and not continue_flag:
                        print(f"Our chunks are too small. Or too many matches start at the same second. Can't proceed.")
                        print(f"Make chunks larger, if possible.")
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.to_csv(f'temporary.csv', index=False)
                        print(f"Partially extracted data is saved to 'temporary.csv', you can continue later from this point.")
                        break_flag = 1
                        break

                    continue_flag = 0
                    n += 1
                    url = f'''https://web.cyanide-studio.com/ws/bb3/matches/?key={api_key}\
&competition_id={competition_id}\
&start={competition_start}\
&end={anchor_date}
&limit={chunk_limit}'''
                    response = re.get(url)
                    
                    if response.status_code == 200:
                        print(f'{n} request: OK')
                        
                    else:
                        print(f'{n} request: ERROR: {response.status_code}')
                    
                    data = response.json()
                    df = pd.json_normalize(data['matches'])
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
                    anchor_date = combined_df.started.min()

                if not break_flag:
                    print(f"Data extracted with {n} requests to 'matches' endpoint.")
                
                    if n >= request_limit:
                        print('Request limit exceeded. Try to continue later.')
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.to_csv(f'temporary.csv', index=False)
                        print(f"Partially extracted data is saved to 'temporary.csv'")
                
                    else:
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.drop('api_match', axis=1, inplace=True)
                        combined_df.info()
                        combined_df.to_csv(f'{competition_name}_{platform}_raw_data.csv', index=False)
                        print(f"Data is saved to '{competition_name}_{platform}_raw_data.csv'")
                        
                        if Path('temporary.csv').exists():
                            Path('temporary.csv').unlink()
                            print(f'temporary.csv deleted.')
                
            except Exception as e:
                print(f"Something broke: {e}. Data partially extracted with {n} requests to 'matches' endpoint.")
                combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                combined_df.to_csv(f'temporary.csv', index=False)
                print(f"Partially extracted data is saved to 'temporary.csv', you can continue later from this point.")
        
        else:
            print("No competition with this ID in this league.")
            
    else:
        print("No competitions in this league.")
        
else:
    print(f'Error: {response.status_code}')

Response: OK
Start extracting data.
1 request: OK
2 request: OK
3 request: OK
4 request: OK
5 request: OK
6 request: OK
7 request: OK
8 request: OK
9 request: OK
10 request: OK
11 request: OK
12 request: OK
13 request: OK
14 request: OK
15 request: OK
16 request: OK
17 request: OK
18 request: OK
19 request: OK
20 request: OK
21 request: OK
22 request: OK
23 request: OK
24 request: OK
25 request: OK
26 request: OK
27 request: OK
28 request: OK
29 request: OK
30 request: OK
31 request: OK
32 request: OK
33 request: OK
34 request: OK
35 request: OK
36 request: OK
37 request: OK
38 request: OK
39 request: OK
40 request: OK
41 request: OK
42 request: OK
43 request: OK
44 request: OK
45 request: OK
46 request: OK
47 request: OK
48 request: OK
49 request: OK
50 request: OK
51 request: OK
52 request: OK
53 request: OK
54 request: OK
55 request: OK
56 request: OK
57 request: OK
58 request: OK
59 request: OK
60 request: OK
61 request: OK
62 request: OK
63 request: OK
64 request: OK
65 request: O

This way we get raw data "as-is", with some cells containing dictionaries. Working with this data format may be inconvenient, so following code will process data to "expand" those sells.

In [27]:
def expand_dict_column(df, column_name):
    """
    Function for "flattening" columns with dictionaries in cells.
    
    :param df: original DataFrame
    :param column_name: column for "expanding"
    :return: new DataFrame with expanded columns
    """
    # Testing if column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")
    
    # Get keys
    keys = df[column_name].iloc[0][0].keys()
    
    # Make new columns out of keys
    for i in range(len(df[column_name].iloc[0])):
        for key in keys:
            new_column_name = f"{key}_{i + 1}"
            df[new_column_name] = df[column_name].apply(lambda x: x[i][key])
    
    # Delete original columns (if not needed)
    df = df.drop(columns=[column_name])
    
    return df

In [28]:
if Path(f'{competition_name}_{platform}_raw_data.csv').exists():

    filename = f'{competition_name}_{platform}_raw_data.csv'
    df = pd.read_csv(filename)
    
    df['coaches'] = df['coaches'].apply(ast.literal_eval)
    df['teams'] = df['teams'].apply(ast.literal_eval)

    df = expand_dict_column(df, 'coaches')
    df = expand_dict_column(df, 'teams')

    df.to_csv(f'{competition_name}_{platform}_processed_data.csv', index=False)
    print(f"File {competition_name}_{platform}_processed_data.csv processed and saved.")

else:
    print("File not found.")

File official_ladder_season_07_crossplay_processed_data.csv processed and saved.


Result is 2 files: {competition_name}\_{platform}\_raw_data.csv with raw data and {competition_name}\_{platform}\_processed_data.csv with processed (more convenient to work with) data.